In [1]:
import pyedflib
import numpy as np
import xmltodict
import json
import mne
import matplotlib
import math
import pathlib
from mne_extras import write_mne_edf

In [2]:
matplotlib.use('Qt5Agg')

In [30]:
#setting up paths for inputs and outputs
file_name = '2021-07-12_111459_RawData'

input_path = './input_txt/mzk_20210629/'
output_path_edf = './output_edf/'
output_path_fif = './output_fif/'

# ch_one = file_name + '_Ch1.txt'
# ch_two = file_name + '_Ch2.txt'
ch_one = '2021-07-12_111459_Filter_Data' + '_Ch1.txt'
ch_two = '2021-07-12_111459_Filter_Data' + '_Ch2.txt'

ch_three = file_name + '_Ch3.txt'
ch_four = file_name + '_Ch4.txt'

xml = '2021-07-12_111459.xml'

out_file_name = '2021-07-12_111459_Filter_Data_me_thinking'
out_name_edf = out_file_name + '.edf'
out_name_fif = out_file_name + '.fif'
out_name_final = 'final_' + out_file_name + '.edf'

ch_one_path = input_path + ch_one
ch_two_path = input_path + ch_two
ch_three_path = input_path + ch_three
ch_four_path = input_path + ch_four
xml_path = input_path + xml
out_file_path_edf = output_path_edf + out_name_edf
out_file_path_fif = output_path_fif + out_name_fif
out_final_edf = output_path_edf + out_name_final

In [31]:
#read from .txt and convert data into numpy array
#each dataset is composed of two channels (.txt) and one information doc (.xml)
raw_one = np.loadtxt(ch_one_path)
raw_two = np.loadtxt(ch_two_path)
raw_three = np.loadtxt(ch_three_path)
raw_four = np.loadtxt(ch_four_path)
signal = [np.array(raw_one, dtype=np.float32), np.array(raw_two, dtype=np.float32), np.array(raw_three, dtype=np.float32), np.array(raw_four, dtype=np.float32)]

In [32]:
#read .xml doc and extract needed info
fileptr = open(xml_path, "r")

xml_content = fileptr.read()

my_ordered_dict = xmltodict.parse(xml_content)
dict = json.loads(json.dumps(my_ordered_dict))

sample_rate = eval(dict['RECORD_INFO']['Record']['SamplesFreq'])

In [33]:
#setting up info needed for .edf generation and write .edf file
headers = [{'label':'ch1', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
            {'label':'ch2', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch3', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'},
          {'label':'ch4', 
            'dimension': 'uV',
            'sample_rate': sample_rate,
            'physical_max': 5000,
            "physical_min": -5000,
            'digital_max': 5000,
            'digital_min': -5000,
            'transducer': 'None',
            'prefilter': 'None'}]
with open(out_file_path_edf, 'w') as output:
    print(out_file_path_edf)
    flag = pyedflib.highlevel.write_edf(output.name, signal, headers, header=None, digital=False, file_type=-1, block_size=1)
    print(flag)

./output_edf/2021-07-12_111459_Filter_Data_me_thinking.edf
True


In [34]:
#read the newly created .edf using mne
raw=mne.io.read_raw_edf(out_file_path_edf,preload=False)

Extracting EDF parameters from C:\Users\admin\Desktop\work\my_evaluator\output_edf\2021-07-12_111459_Filter_Data_me_thinking.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [35]:
raw

<RawEDF | 2021-07-12_111459_Filter_Data_me_thinking.edf, 4 x 40960 (40.0 s), ~9 kB, data not loaded>

In [36]:
raw.plot()

<MNEBrowseFigure size 1919x786 with 4 Axes>

Channels marked as bad: ['ch1']


In [37]:
#create events using mne
#events are equally spaced out for epoch division
new_events = mne.make_fixed_length_events(raw, duration=2.)
event_dict = {'divide':1}

In [38]:
#reject using values of amplitude
#best: do before the filter, but will reject 89% of data
del (dict)     
epochs = mne.Epochs(raw,new_events, reject_by_annotation=False, preload=True)

Not setting metadata
Not setting metadata
20 matching events found
Setting baseline interval to [-0.2001953125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 20 events and 718 original time points ...
1 bad epochs dropped


In [39]:
epochs_2 = mne.Epochs(raw,new_events, reject_by_annotation=False, preload=True)

Not setting metadata
Not setting metadata
20 matching events found
Setting baseline interval to [-0.2001953125, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 20 events and 718 original time points ...
1 bad epochs dropped


In [40]:
epochs.plot_psd(average=True)

    Using multitaper spectrum estimation with 7 DPSS windows


<MNELineFigure size 1000x350 with 1 Axes>

In [41]:
epochs.plot()
epochs_2.plot()

<MNEBrowseFigure size 1919x786 with 4 Axes>

Dropped 0 epochs: 
Channels marked as bad: ['ch1']
Dropped 0 epochs: 
Channels marked as bad: ['ch1']


In [42]:
#save as .fif format
epochs.save(out_file_path_fif, overwrite=True)

C:\Users\admin\AppData\Local\Temp/ipykernel_7188/2457816835.py:2: RuntimeWarning: This filename (./output_fif/2021-07-12_111459_Filter_Data_me_thinking.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(out_file_path_fif, overwrite=True)


In [43]:
#convert to data frame format in order to save as .edf
df = epochs.to_data_frame()

In [44]:
#save as .edf
out_raw_one = df['ch1']
out_raw_two = df['ch2']
out_raw_three = df['ch3']
out_raw_four = df['ch4']
out_signal = [np.array(out_raw_one), np.array(out_raw_two), np.array(out_raw_three), np.array(out_raw_four)]

In [45]:
variance = np.var(out_signal)
print('variance:', variance)


variance: 981.1691658211894


In [46]:
epochs.plot_psd(average=True)

    Using multitaper spectrum estimation with 7 DPSS windows


<MNELineFigure size 1000x350 with 1 Axes>

In [47]:
psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs, fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_power, frqs_power = mne.time_frequency.psd_multitaper(epochs, fmin=45, fmax=55, tmin=None, tmax=None)
power_sum_pds = np.sum(psds_power)
print(power_sum_pds)
power_ratio = (power_sum_pds) / (total_sum_pds)
print(power_ratio)
print(frqs_power)

    Using multitaper spectrum estimation with 7 DPSS windows
2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
2.5071463598825664e-05
0.8796783608027269
[45.63788301 47.06406685 48.4902507  49.91643454 51.34261838 52.76880223
 54.19498607]


In [48]:
# psds_total, frqs_total = mne.time_frequency.psd_welch(epochs, fmin=0.1, fmax=100, tmin=None, tmax=None, n_fft = 718)
# total_sum_pds = np.sum(psds_total)
# print(total_sum_pds)
# psds_part, frqs_part = mne.time_frequency.psd_welch(epochs, fmin=45, fmax=55, tmin=None, tmax=None, n_fft = 718)
# part_sum_pds = np.sum(psds_part)
# print(part_sum_pds)
# ratio = (part_sum_pds) / (total_sum_pds)
# print(ratio)
# print(frqs_part)

In [49]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#delta
print(total_sum_pds)
psds_delta, frqs_delta = mne.time_frequency.psd_multitaper(epochs, fmin=1.5, fmax=4.5, tmin=None, tmax=None)
delta_sum_pds = np.sum(psds_delta)
print(delta_sum_pds)
delta_ratio = (delta_sum_pds) / (total_sum_pds)
print(delta_ratio)
print(frqs_delta)

2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
1.0387817245660125e-06
0.036447565220758045
[2.85236769 4.27855153]


In [50]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#theta
print(total_sum_pds)
psds_theta, frqs_theta = mne.time_frequency.psd_multitaper(epochs, fmin=4.5, fmax=7.5, tmin=None, tmax=None)
theta_sum_pds = np.sum(psds_theta)
print(theta_sum_pds)
theta_ratio = (theta_sum_pds) / (total_sum_pds)
print(theta_ratio)
print(frqs_theta)

2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
5.026525902656933e-07
0.017636489585669064
[5.70473538 7.13091922]


In [51]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#alpha
print(total_sum_pds)
psds_alpha, frqs_alpha = mne.time_frequency.psd_multitaper(epochs, fmin=7.5, fmax=12.5, tmin=None, tmax=None)
alpha_sum_pds = np.sum(psds_alpha)
print(alpha_sum_pds)
alpha_ratio = (alpha_sum_pds) / (total_sum_pds)
print(alpha_ratio)
print(frqs_alpha)

2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
1.8685383165754102e-07
0.006556109965192197
[ 8.55710306  9.98328691 11.40947075]


In [52]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#beta
print(total_sum_pds)
psds_beta, frqs_beta = mne.time_frequency.psd_multitaper(epochs, fmin=14.5, fmax=29.5, tmin=None, tmax=None)
beta_sum_pds = np.sum(psds_beta)
print(beta_sum_pds)
beta_ratio = (beta_sum_pds) / (total_sum_pds)
print(beta_ratio)
print(frqs_beta)

2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
2.7286507581920147e-07
0.009573972483528644
[15.68802228 17.11420613 18.54038997 19.96657382 21.39275766 22.8189415
 24.24512535 25.67130919 27.09749304 28.52367688]


In [53]:
#delta=(2, 4), theta=(5, 7), alpha=(8, 12), beta=(15, 29), gamma=(30, 45))
#gamma
psds_total, frqs_total = mne.time_frequency.psd_multitaper(epochs, fmin=0.5, fmax=55, tmin=None, tmax=None)
total_sum_pds = np.sum(psds_total)
print(total_sum_pds)
psds_gamma, frqs_gamma = mne.time_frequency.psd_multitaper(epochs, fmin=29.5, fmax=45.5, tmin=None, tmax=None)
gamma_sum_pds = np.sum(psds_gamma)
print(gamma_sum_pds)
gamma_ratio = (gamma_sum_pds) / (total_sum_pds)
print(gamma_ratio)
print(frqs_gamma)

    Using multitaper spectrum estimation with 7 DPSS windows
2.8500716530013733e-05
    Using multitaper spectrum estimation with 7 DPSS windows
6.980195530679918e-07
0.024491298397109295
[29.94986072 31.37604457 32.80222841 34.22841226 35.6545961  37.08077994
 38.50696379 39.93314763 41.35933148 42.78551532 44.21169916]


In [54]:
#evaluate the signal

#formula from paper

#y1: variance score
y1 = 0
#x1: variance
x1 = variance

if x1 < 50:
    y1 = 0.02*np.power(x1,2)
elif x1 >= 50 and x1 < 100:
    y1 = 0.6*x1 + 20
elif x1 >= 100 and x1 < 2000:
    y1 = 100
elif x1 >= 2000 and x1 < 5000:
    y1 = -0.013333*x1 + 126.6
elif x1 >= 5000 and x1 < 10000:
    y1 = 0.006*x1 + 90
else:
    y1 = 15/0.02*np.power((x1-10000),2)
    
#y2: power voltage score
y2 = 0
#x2: signal that are 50Hz (CN) / Total
x2 = power_ratio

if x2 < 0.01:
    y2 = 1
elif x2 >= 0.01 and x2 < 0.1:
    y2 = 1 - 24.691*np.power((x2-0.01),2)
elif x2 >= 0.1 and x2 < 5:
    y2 = 0.8 - 0.00833*np.power((x2-0.1),2)
elif x2 >= 5 and x2 < 10:
    y2 = 0.9 - np.power(0.06,2)
else:
    y2 = 30 / np.power(x2,2)
    
    
#y3: alpha score
y3 = 0
#x3: alpha wave / Total
x3 = alpha_ratio

if x3 < 0.5:
    y3 = 2.8 * np.power(x3,2)
else:
    y3 = 1.2 * np.power(x3,2) + 2.4*x3 - 0.2

print('x1: ' + str(x1))
print('x2: ' + str(x2))
print('x3: ' + str(x3))

print('y1: ' + str(y1))
print('y2: ' + str(y2))
print('y3: ' + str(y3))

print('y1*y2*y3: ' + str(y1*y2*y3))

x1: 981.1691658211894
x2: 0.8796783608027269
x3: 0.006556109965192197
y1: 100
y2: 0.7949362067752875
y3: 0.0001203512180519388
y1*y2*y3: 0.009567154075899373


In [29]:
print(1*2*3)

6
